In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git@1870fb05a903546b79236d277ae4bc12e626b328 transformers accelerate imageio[ffmpeg]

In [ ]:
!pip install -q opencv-contrib-python
!pip install -q controlnet_aux
!pip install -q -U xformers

In [1]:
import diffusers
import transformers
print(diffusers.__version__, transformers.__version__)
# try:
#     import xformers
#     print(xformers.__version__)
# except:
#     print("xformers not installed correctly")

c:\Users\chris\.conda\envs\txt2vid\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 1.13.1+cu117 with CUDA 1107 (you have 2.0.0+cu117)
    Python  3.9.13 (you have 3.9.16)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


0.15.0.dev0 4.27.3


In [9]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler, StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import HTML
from base64 import b64encode
import cv2
from PIL import Image
import numpy as np

import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def dummy(images, **kwargs):
    return images, False

pipe = DiffusionPipeline.from_pretrained("./diffusion_pipeline/", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()
pipe.safety_checker = dummy

controlnet = [
    ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose", torch_dtype=torch.float16),
    ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16),
]
controlpipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)
controlpipe.scheduler = UniPCMultistepScheduler.from_config(controlpipe.scheduler.config)
controlpipe.enable_model_cpu_offload()
controlpipe.safety_checker = dummy
#controlpipe.enable_xformers_memory_efficient_attention()

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [11]:
# pipe.save_pretrained("./diffusion_pipeline/")
# controlpipe.save_pretrained("./controlnet_pipeline/")

NotImplementedError: Currently, the `save_pretrained()` is not implemented for Multi-ControlNet.

In [40]:
prompt = 'a beautiful woman dancing in a field, best quality, masterpiece, photorealistic, unreal engine 5'
negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality, bad face,"
video_duration_seconds = 3
num_frames = video_duration_seconds * 10
video_frames = pipe(prompt, negative_prompt=negative_prompt, num_inference_steps=25, num_frames=num_frames).frames
video_path = export_to_video(video_frames)

100%|██████████| 25/25 [00:18<00:00,  1.32it/s]


In [41]:
def display_video(video):
    fig = plt.figure(figsize=(4.2,4.2))  #Display size specification
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=100, repeat_delay=1000)

    plt.close()
    return anime
video = imageio.mimread(video_path)  #Loading video
writer = imageio.get_writer('./videov1.mp4', fps=10)
for frame in video:
    writer.append_data(frame)
writer.close()

In [42]:
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image

for i in range(len(video_frames)):
    image = np.array(video_frames[i])

    low_threshold = 100
    high_threshold = 200

    image = cv2.Canny(image, low_threshold, high_threshold)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    canny_image = Image.fromarray(image)
    #canny_image.show()
    openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")
    openpose_image = openpose(video_frames[i])
    
    images = [openpose_image, canny_image]
    output = controlpipe(
        'Russel Brand dancing on the SNL stage, best quality, masterpiece, photorealistic, unreal engine 5',
        images,
        negative_prompt=negative_prompt,
        generator=torch.Generator(device='cpu'),
        num_inference_steps=20,
    )
    output.images[0].save(f'./temp_imgs/controlnet-{i}.png')

cuda


100%|██████████| 20/20 [00:03<00:00,  5.02it/s]


cuda


100%|██████████| 20/20 [00:03<00:00,  5.01it/s]


cuda


...

100%|██████████| 20/20 [00:04<00:00,  4.95it/s]


cuda


100%|██████████| 20/20 [00:04<00:00,  4.92it/s]


cuda


100%|██████████| 20/20 [00:04<00:00,  4.92it/s]


In [43]:
from pathlib import Path

writer = imageio.get_writer('./videov2.mp4', fps=10)

for file in Path("./temp_imgs/").iterdir():
    if not file.is_file():
        continue
    im = imageio.imread(file)
    writer.append_data(im)
writer.close()

C:\Users\chris\AppData\Local\Temp\ipykernel_5280\3214315076.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imageio.imread(file)
